In [4]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
from datetime import datetime

In [11]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
from datetime import datetime


# Define the URL
url = 'https://www.tsinghua.edu.cn/news/spqh/jctj.htm'

driver = webdriver.Safari()

# Open the web page in the driver
driver.get(url)


# Extract the HTML from the web page
html = driver.page_source

# Parse the HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')

# Define a list to store the extracted data
df = pd.DataFrame(columns=['Title', 'Date', 'URL', 'Description'])

while True:
    try:
        # Extract the HTML from the web page
        html = driver.page_source

        # Parse the HTML with Beautiful Soup
        soup = BeautifulSoup(html, 'html.parser')

        # Extract the data from the soup object
        video_title = soup.find_all('p', class_="bt")[1:9]
        titles = [vedio.text.strip() for vedio in video_title]

        # Find the <ul> tag with class "qhrw2_ul"
        ul_tag = soup.find('ul', class_='qhrw2_ul')
        # Find all <a> tags within the <ul> tag and get their href attributes     
        links = [urljoin(url, a_tag.get('href')) for a_tag in ul_tag.find_all('a')]

        # Find all <div> tags with class "sj"
        div_tags = ul_tag.find_all('div', class_='sj')
        # Loop through each <div> tag and get the date
        dates = []
        for div_tag in div_tags:
            p_tag = div_tag.find('p')
            span_tag = div_tag.find('span')
            date_str = f"{p_tag.text}/{span_tag.text}"
            date = datetime.strptime(date_str, '%d/%Y.%m')
            dates.append(date.strftime('%Y/%m/%d'))

        # For the description    
        words = soup.find_all('p', class_='zy')[1:]
        word = [word.text.strip() for word in words]

        # Ensure that all the lists in the dictionary have the same length
        num_rows = min(len(titles), len(dates), len(links), len(word))
        new_data = {'Title': titles[:num_rows], 'Date': dates[:num_rows], 'URL': links[:num_rows], 'Description': word[:num_rows]}

        # Convert the dictionary to a Pandas DataFrame
        new_df = pd.DataFrame(new_data)

        # Append the new DataFrame to the original DataFrame
        df = pd.concat([df, new_df], ignore_index=True)

        # Wait for the next button to become clickable
        wait = WebDriverWait(driver, 10)
        next_link = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@class='next']")))

        # Check if the next link has a valid href attribute
        if next_link.get_attribute('href') == "javascript:;":
            # If the href attribute is "javascript:;", it means that we have reached the last page
            break
        else:
            # Click on the next button
            next_link.click()

    except StaleElementReferenceException:
        # If the element reference is stale, locate the element again
        pass

    except TimeoutException:
        # If the "next" button is not clickable, it means that we have reached the last page
        break



# Extract data from the first featured post
video_title = soup.find_all('p', class_="bt")[0]
titles = [vedio.text.strip() for vedio in video_title]

ul_tag = soup.find('div', class_='qhrw2_first')
links = [urljoin(url, a_tag.get('href')) for a_tag in ul_tag.find_all('a')]

ul_tag = soup.find('div', class_='sj')
dates = []
p_tag = ul_tag.find('p')
span_tag = ul_tag.find('span')
date_str = f"{p_tag.text}/{span_tag.text}"
date = datetime.strptime(date_str, '%d/%Y.%m')
dates.append(date.strftime('%Y/%m/%d'))

words = soup.find_all('p', class_='zy')[0]
word = [word.text.strip() for word in words]

new_data = {'Title': titles, 'Date': dates, 'URL': links, 'Description': word}
new_df = pd.DataFrame(new_data)
df = pd.concat([df, new_df], ignore_index=True)

# Close the driver
driver.quit()

df.drop_duplicates(inplace=True)

# Export the DataFrame to a CSV file
df.to_csv(r'/Users/davidsmacbook/Desktop/Work/Data Scraping from THU.csv', index=False)

# Print a message to confirm the export
print('Data exported to data.csv')

Data exported to data.csv


In [3]:
driver.quit()

NameError: name 'driver' is not defined

In [51]:
# First Featured post
video_title = soup.find_all('p', class_ ="bt")[0]
titles = [vedio.text.strip() for vedio in video_title]
titles

['【2023毕业故事】李润凤：去支教 让梦想在西部绽放']

In [64]:
# Find the <ul> tag with class "qhrw2_first"

ul_tag = soup.find('div', class_='qhrw2_first')
links = [a_tag.get('href') for a_tag in ul_tag.find_all('a')]

links


['../../info/2061/105129.htm']

In [92]:
# Find the <ul> tag with class "qhrw2_ul"
ul_tag = soup.find('div', class_='sj')
dates = []
p_tag = ul_tag.find('p')
span_tag = ul_tag.find('span')
date_str = f"{p_tag.text}/{span_tag.text}"
date = datetime.strptime(date_str, '%d/%Y.%m')
dates.append(date.strftime('%Y/%m/%d'))


dates


['2023/06/30']

In [80]:
words = soup.find_all('p', class_='zy')[0]

word = [word.text.strip() for word in words]

word

['在清华的4年，李润凤用攀登这个关键词来概括自己的成长，不断地去爬台阶，从0走向了无限可能。如今完成本科学业的她，毕业后将前往云南省南涧彝族自治县支教，到祖国和人民需要的地方去，为实现高质量发展，为实现中华民族伟大复兴去贡献一份青春力量。']

In [50]:
video_title = soup.find_all('p', class_ ="bt")[1:9]
titles = [vedio.text.strip() for vedio in video_title]

titles


['【2023毕业故事】投身基层服务！清华“励志学子”林万东毕业回家乡',
 '【2023毕业故事】代宗仁：我和家乡有个约定',
 '【2023毕业故事】去祖国西部当老师！轮椅上的清华博士毕业了',
 '【2023毕业故事】潘长城：投身国家水安全保障事业',
 '加入为先书院，成为定义未来的科技领导者！',
 '【秀钟书院招生宣传片】水木清华众秀钟',
 '校党委书记邱勇一行来到清华附小，与师生共庆“六一”',
 '学习贯彻习近平新时代中国特色社会主义思想主题教育专题研修主题报告会']

In [47]:
# Find the <ul> tag with class "qhrw2_ul"
ul_tag = soup.find('ul', class_='qhrw2_ul')

# Find all <a> tags within the <ul> tag and get their href attributes
links = [a_tag.get('href') for a_tag in ul_tag.find_all('a')]

#links = [link.get('herf') for link in soup.find_all('ul', class_='qhrw2_ul')]

links



['../../info/2061/105058.htm',
 '../../info/2061/104987.htm',
 '../../info/2061/104962.htm',
 '../../info/2061/104958.htm',
 '../../info/2061/104884.htm',
 '../../info/2061/104882.htm',
 '../../info/2061/104705.htm',
 '../../info/2061/104022.htm']

In [90]:


# Find the <ul> tag with class "qhrw2_ul"
ul_tag = soup.find('ul', class_='qhrw2_ul')

# Find all <div> tags with class "sj"
div_tags = ul_tag.find_all('div', class_='sj')

# Loop through each <div> tag and get the date
dates = []
for div_tag in div_tags:
    p_tag = div_tag.find('p')
    span_tag = div_tag.find('span')
    date_str = f"{p_tag.text}/{span_tag.text}"
    date = datetime.strptime(date_str, '%d/%Y.%m')
    dates.append(date.strftime('%Y/%m/%d'))

# Print the list of dates
print(dates)


['2023/06/30', '2023/06/27', '2023/06/25', '2023/06/24', '2023/06/24', '2023/06/20', '2023/06/20', '2023/06/02', '2023/05/21']


In [79]:
words = soup.find_all('p', class_='zy')[1:]

word = [word.text.strip() for word in words]

word

['四年前，正在工地搬砖的云南考生林万东收到了来自清华大学的录取通知书。如今，顺利完成本科学业的他，选择回到家乡投身基层公共部门工作。他说，“希望自己化作一颗小小的‘炭火’，无声燃烧，却可以传递温暖。”万东，加油！',
 '“回家乡去！投身国防军工事业！”当毕业去哪这一重要抉择来临之际，清华大学精仪系博士研究生代宗仁坚定地给出了他的答案。他希望自己向下扎根、向上成长，“做一名快乐生活、阳光热血的新青年，把自己的一生写进祖国的建设历史中。”',
 '17岁时，他以优异的成绩考入清华大学，曾经的运动达人却在大一下学期突发脊髓血管瘤全身瘫痪。清华“自强不息”的精神激励着他一点点重拾信心，坐上轮椅回到校园后，他以惊人的毅力顺利完成了本科、博士阶段的学业，并选择去祖国西部从事教书育人和科研工...',
 '又到了凤凰花开的季节，毕业季如期而至。从校门到社会，清华园里莘莘学子即将奔赴人生的下一站。当“毕业去哪儿”这一重要抉择来临之际，清华大学水利水电工程系博士研究生潘长城，坚定地给出了他的答案：投身国家水安全保障事业，践行清华人的使命与担当...',
 '为先书院作为清华大学创新工科的先行示范，探索科技创新领军人才培养的新模式。 为先书院以为国家培养未来科技创新领军人才为使命任务，构筑“宽厚基础-工程实践-探索研究”三要素融合的培养体系、“师生从游、滴灌培育”的教学模式。为先书院注重学生过程...',
 '为满足绿色低碳发展战略需求，破解全球可持续发展重大命题，清华大学成立秀钟书院，并于今年开始招生。如果你心怀地球与人类命运，学术志趣浓厚，创新潜力突出，欢迎加入秀钟书院！',
 '在“六一”国际儿童节到来之际，校党委书记邱勇一行来到清华附小，与师生共庆“六一”，向大家致以节日祝贺，向辛勤工作的附小教师表示诚挚问候。',
 '在主题教育如火如荼开展之时，5月20日上午，新清华学堂再掀热潮。作为清华大学学习贯彻习近平新时代中国特色社会主义思想主题教育读书班特色活动之一，校党委书记邱勇以“为以中国式现代化全面推进中华民族伟大复兴贡献清华力量”为题作专题研修主题报告。']

In [5]:
# Extract the data from the soup object
video_title = soup.find_all('div', class_ ="item-title04 item-title05")
titles = [vedio.text.strip() for vedio in video_title]

dates = soup.find_all('span', class_ ="item-date")[3:]
date = [vedio.text.strip() for vedio in dates]

links = soup.find_all('source')
link = [vediolink['src'] for vediolink in links][3:]

Words =
word

SyntaxError: invalid syntax (3143688150.py, line 11)

In [10]:
driver.quit()